In [42]:
### IMPORT BIBLIOTECAS

import requests
import re
import matplotlib
import matplotlib.pyplot as plt
import seaborn  as sns
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import folium
import json
from tqdm.auto import tqdm

In [4]:
# Resquest no site da ANAC
request = requests.get('https://www.anac.gov.br/assuntos/dados-e-estatisticas/historico-de-voos')
request

<Response [200]>

In [26]:
# Obtendo links para acessar cada mês de voos
soup = BeautifulSoup(request.content)
a = soup.find_all('a', attrs = {'class':'internal-link'})
links = [i['href'] for i in a if '2019' in i['href']]

In [101]:
# Extraindo csv dos links e unindo em única base de dados
# o try e except é necessário neste caso pois um dos arquivos é separado
# por '\t', causando erro ao tentar separar por ';'

dfs =[]
for i in tqdm(range(len(links))):
    try:
        x_i = pd.read_csv(links[i], sep = ';', encoding = 'mbcs')
        x_i.columns = ['sg_empresa_icao', 'nr_voo', 'cd_di', 'cd_tipo_linha', 'sg_icao_origem',
           'sg_icao_destino', 'dt_partida_prevista', 'dt_partida_real',
           'dt_chegada_prevista', 'dt_chegada_real', 'situacao',
           'cd_justificativa']
        dfs.append(x_i)
    except:
        x_i = pd.read_csv(links[i], sep = '\\t', encoding = 'mbcs')
        x_i.columns = ['sg_empresa_icao', 'nr_voo', 'cd_di', 'cd_tipo_linha', 'sg_icao_origem',
           'sg_icao_destino', 'dt_partida_prevista', 'dt_partida_real',
           'dt_chegada_prevista', 'dt_chegada_real', 'situacao',
           'cd_justificativa']
        dfs.append(x_i)        
data = pd.concat(dfs)

C:\Users\Matheus\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Matheus\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  del sys.path[0]


In [108]:
data = data.reset_index(drop = True)
data

,sg_empresa_icao,nr_voo,cd_di,cd_tipo_linha,sg_icao_origem,sg_icao_destino,dt_partida_prevista,dt_partida_real,dt_chegada_prevista,dt_chegada_real,situacao,cd_justificativa
0,AAF,35,0,I,LFPO,SBKP,25/01/2019 06:15,25/01/2019 06:15,25/01/2019 18:15,25/01/2019 18:15,REALIZADO,NaN
1,AAF,35,0,I,LFPO,SBKP,27/01/2019 06:15,27/01/2019 06:15,27/01/2019 18:15,27/01/2019 18:15,REALIZADO,NaN
2,AAF,35,0,I,LFPO,SBKP,29/01/2019 06:15,29/01/2019 06:15,29/01/2019 18:15,29/01/2019 18:15,REALIZADO,NaN
3,AAF,36,0,I,SBKP,LFPO,25/01/2019 20:15,25/01/2019 20:15,26/01/2019 07:45,26/01/2019 07:45,REALIZADO,NaN
4,AAF,36,0,I,SBKP,LFPO,27/01/2019 20:15,27/01/2019 20:15,28/01/2019 07:45,28/01/2019 07:45,REALIZADO,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
982986,GLO,1652,0,N,SBSP,SBIL,25/12/2019 10:45,25/12/2019 10:54,25/12/2019 12:50,25/12/2019 13:02,REALIZADO,NaN
982987,GLO,1633,0,N,SBPL,SBGR,31/12/2019 02:45,31/12/2019 02:42,31/12/2019 05:30,31/12/2019 05:39,REALIZADO,NaN
982988,GLO,2013,0,N,SBNF,SBGL,06/12/2019 18:55,06/12/2019 19:32,06/12/2019 20:20,06/12/2019 20:52,REALIZADO,RI
982989,GLO,4034,0,N,SBRJ,SBBR,10/12/2019 17:00,10/12/2019 17:03,10/12/2019 18:50,10/12/2019 18:43,REALIZADO,NaN


In [109]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 982991 entries, 0 to 982990
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   sg_empresa_icao      982991 non-null  object
 1   nr_voo               982699 non-null  object
 2   cd_di                982754 non-null  object
 3   cd_tipo_linha        982991 non-null  object
 4   sg_icao_origem       982991 non-null  object
 5   sg_icao_destino      982991 non-null  object
 6   dt_partida_prevista  982976 non-null  object
 7   dt_partida_real      965183 non-null  object
 8   dt_chegada_prevista  982976 non-null  object
 9   dt_chegada_real      965183 non-null  object
 10  situacao             982991 non-null  object
 11  cd_justificativa     330153 non-null  object
dtypes: object(12)
memory usage: 90.0+ MB


In [111]:
data.describe().T

,count,unique,top,freq
sg_empresa_icao,982991,63,AZU,302316
nr_voo,982699,11832,248,1049
cd_di,982754,22,0,498465
cd_tipo_linha,982991,7,N,814385
sg_icao_origem,982991,361,SBGR,138257
sg_icao_destino,982991,362,SBGR,138586
dt_partida_prevista,982976,125650,20/12/2019 06:00,43
dt_partida_real,965183,394030,20/08/2019 06:00,20
dt_chegada_prevista,982976,125594,06/12/2019 20:25,29
dt_chegada_real,965183,403532,28/10/2019 20:25,18


In [106]:
# pegando uma tabela com os códigos de todos os aeroportos (ANAC), assim será possível adicionar o nome das cidades ao dataframe

icao = pd.read_excel("https://www.anac.gov.br/assuntos/dados-e-estatisticas/vra/glossario_de_aerodromo.xls", encoding ='latin-1', skiprows = 3)
icao.head()

,Unnamed: 0,Sigla OACI,Descrição,Cidade,UF,País,Continente
0,NaN,1AON,OUROLANDIA DO NORTE,OURILANDIA DO NORTE,PA,BRASIL,AMERICA DO SUL
1,NaN,2NHT,HOTEL TRANSAMÉRICA,UNA,BA,BRASIL,AMERICA DO SUL
2,NaN,6ASO,SORRISO,SORRISO,MT,BRASIL,AMERICA DO SUL
3,NaN,CYMX,MONTREAL-MIRABEL INTERNATIONAL AIRPORT,MIRABEL,NaN,CANADA,AMERICA DO NORTE
4,NaN,CYQB,QUEBEC CITY JEAN LESAGE INTERNATIONAL AIRPORT,QUEBEC JEAN LESAGE INTL,NaN,CANADA,AMERICA DO NORTE


In [112]:
# criando dicionario ICAO:cidade

icaodict = {}
for i in range(len(icao)):
    icaodict.update({icao.loc[i]['Sigla OACI']:icao.loc[i]['Cidade']})
    
# criando dicionario CIDADE:PAÌS
paisdict = {}
for i in range(len(icao)):
    paisdict.update({icao.loc[i]['Cidade']:icao.loc[i]['País']})

In [116]:
# adicionando novas colunas ao dataframe
data['cidade_origem'] = data['sg_icao_origem'].apply(lambda x: icaodict.get(x))
data['cidade_destino'] = data['sg_icao_destino'].apply(lambda x: icaodict.get(x))
data['pais_origem'] = data['cidade_origem'].apply(lambda x: paisdict.get(x))
data['pais_destino'] = data['cidade_destino'].apply(lambda x: paisdict.get(x))
data

,sg_empresa_icao,nr_voo,cd_di,cd_tipo_linha,sg_icao_origem,sg_icao_destino,dt_partida_prevista,dt_partida_real,dt_chegada_prevista,dt_chegada_real,situacao,cd_justificativa,cidade_origem,cidade_destino,pais_origem,pais_destino
0,AAF,35,0,I,LFPO,SBKP,25/01/2019 06:15,25/01/2019 06:15,25/01/2019 18:15,25/01/2019 18:15,REALIZADO,NaN,PARIS,CAMPINAS,FRANCA,BRASIL
1,AAF,35,0,I,LFPO,SBKP,27/01/2019 06:15,27/01/2019 06:15,27/01/2019 18:15,27/01/2019 18:15,REALIZADO,NaN,PARIS,CAMPINAS,FRANCA,BRASIL
2,AAF,35,0,I,LFPO,SBKP,29/01/2019 06:15,29/01/2019 06:15,29/01/2019 18:15,29/01/2019 18:15,REALIZADO,NaN,PARIS,CAMPINAS,FRANCA,BRASIL
3,AAF,36,0,I,SBKP,LFPO,25/01/2019 20:15,25/01/2019 20:15,26/01/2019 07:45,26/01/2019 07:45,REALIZADO,NaN,CAMPINAS,PARIS,BRASIL,FRANCA
4,AAF,36,0,I,SBKP,LFPO,27/01/2019 20:15,27/01/2019 20:15,28/01/2019 07:45,28/01/2019 07:45,REALIZADO,NaN,CAMPINAS,PARIS,BRASIL,FRANCA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
982986,GLO,1652,0,N,SBSP,SBIL,25/12/2019 10:45,25/12/2019 10:54,25/12/2019 12:50,25/12/2019 13:02,REALIZADO,NaN,SÃO PAULO,ILHÉUS,BRASIL,BRASIL
982987,GLO,1633,0,N,SBPL,SBGR,31/12/2019 02:45,31/12/2019 02:42,31/12/2019 05:30,31/12/2019 05:39,REALIZADO,NaN,PETROLINA,GUARULHOS,BRASIL,BRASIL
982988,GLO,2013,0,N,SBNF,SBGL,06/12/2019 18:55,06/12/2019 19:32,06/12/2019 20:20,06/12/2019 20:52,REALIZADO,RI,NAVEGANTES,RIO DE JANEIRO,BRASIL,BRASIL
982989,GLO,4034,0,N,SBRJ,SBBR,10/12/2019 17:00,10/12/2019 17:03,10/12/2019 18:50,10/12/2019 18:43,REALIZADO,NaN,RIO DE JANEIRO,BRASÍLIA,BRASIL,BRASIL


In [118]:
# # pegando uma tabela com os códigos de todas as empresas aereas (ANAC), assim será possível adicionar o nome das empresas ao dataframe


empresas = pd.read_excel("https://www.anac.gov.br/assuntos/dados-e-estatisticas/vra/glossario_de_empresas_aereas.xls", encoding ='latin-1', skiprows = 3)
empresas

#criando dicionario CODDIGOEMPRESA:NOMEEMPRESA
empdict = {}
for i in range(len(empresas)):
    empdict.update({empresas.loc[i]['Sigla OACI']:empresas.loc[i]['Nome Empresas']})
    
empdict

# criando nova coluna no datafframe
data['empresa_aerea'] = data['sg_empresa_icao'].apply(lambda x: empdict.get(x))
data

,sg_empresa_icao,nr_voo,cd_di,cd_tipo_linha,sg_icao_origem,sg_icao_destino,dt_partida_prevista,dt_partida_real,dt_chegada_prevista,dt_chegada_real,situacao,cd_justificativa,cidade_origem,cidade_destino,pais_origem,pais_destino,empresa_aerea
0,AAF,35,0,I,LFPO,SBKP,25/01/2019 06:15,25/01/2019 06:15,25/01/2019 18:15,25/01/2019 18:15,REALIZADO,NaN,PARIS,CAMPINAS,FRANCA,BRASIL,None
1,AAF,35,0,I,LFPO,SBKP,27/01/2019 06:15,27/01/2019 06:15,27/01/2019 18:15,27/01/2019 18:15,REALIZADO,NaN,PARIS,CAMPINAS,FRANCA,BRASIL,None
2,AAF,35,0,I,LFPO,SBKP,29/01/2019 06:15,29/01/2019 06:15,29/01/2019 18:15,29/01/2019 18:15,REALIZADO,NaN,PARIS,CAMPINAS,FRANCA,BRASIL,None
3,AAF,36,0,I,SBKP,LFPO,25/01/2019 20:15,25/01/2019 20:15,26/01/2019 07:45,26/01/2019 07:45,REALIZADO,NaN,CAMPINAS,PARIS,BRASIL,FRANCA,None
4,AAF,36,0,I,SBKP,LFPO,27/01/2019 20:15,27/01/2019 20:15,28/01/2019 07:45,28/01/2019 07:45,REALIZADO,NaN,CAMPINAS,PARIS,BRASIL,FRANCA,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
982986,GLO,1652,0,N,SBSP,SBIL,25/12/2019 10:45,25/12/2019 10:54,25/12/2019 12:50,25/12/2019 13:02,REALIZADO,NaN,SÃO PAULO,ILHÉUS,BRASIL,BRASIL,GOL
982987,GLO,1633,0,N,SBPL,SBGR,31/12/2019 02:45,31/12/2019 02:42,31/12/2019 05:30,31/12/2019 05:39,REALIZADO,NaN,PETROLINA,GUARULHOS,BRASIL,BRASIL,GOL
982988,GLO,2013,0,N,SBNF,SBGL,06/12/2019 18:55,06/12/2019 19:32,06/12/2019 20:20,06/12/2019 20:52,REALIZADO,RI,NAVEGANTES,RIO DE JANEIRO,BRASIL,BRASIL,GOL
982989,GLO,4034,0,N,SBRJ,SBBR,10/12/2019 17:00,10/12/2019 17:03,10/12/2019 18:50,10/12/2019 18:43,REALIZADO,NaN,RIO DE JANEIRO,BRASÍLIA,BRASIL,BRASIL,GOL


In [129]:
# pegando uma tabela com as coordenadas de todos os aeroportos (ANAC), assim será possível adicionar as coordenadas dos aeroportos ao dataframe
# visualizando a tabela de coordenadas

airports = pd.read_csv("https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat", header = None)
airports.columns = ['id','aeroporto','cidade','país','cod_aero','icao','lat','lon','a','b','c','d','e','f']
airports[airports['país'].isin(['Brazil'])].head()

,id,aeroporto,cidade,país,cod_aero,icao,lat,lon,a,b,c,d,e,f
2392,2518,Conceição do Araguaia Airport,Conceicao Do Araguaia,Brazil,CDJ,SBAA,-8.348350,-49.301498,653,-3,S,America/Belem,airport,OurAirports
2393,2519,Campo Délio Jardim de Mattos Airport,Rio De Janeiro,Brazil,\N,SBAF,-22.875099,-43.384701,110,-3,S,America/Sao_Paulo,airport,OurAirports
2394,2520,Amapá Airport,Amapa,Brazil,\N,SBAM,2.077510,-50.858200,45,-3,S,America/Fortaleza,airport,OurAirports
2395,2521,Araraquara Airport,Araracuara,Brazil,AQA,SBAQ,-21.812000,-48.132999,2334,-3,S,America/Sao_Paulo,airport,OurAirports
2396,2522,Santa Maria Airport,Aracaju,Brazil,AJU,SBAR,-10.984000,-37.070301,23,-3,S,America/Fortaleza,airport,OurAirports


In [ ]:
# criando dicionario ICAO:LATITUDE
latdict = {}
for i in range(len(airports)):
    latdict.update({airports.loc[i]['icao']:airports.loc[i]['lat']})
    
latdict

#criando novas colunas no dataframe
data['latitude_origem'] = data['sg_icao_origem'].apply(lambda x: latdict.get(x))
data['latitude_destino'] = data['ICAO Aeródromoo Destino'].apply(lambda x: latdict.get(x))


# criando dicionario ICAO:LONGITUDE

londict = {}
for i in range(len(airports)):
    londict.update({airports.loc[i]['icao']:airports.loc[i]['lon']})
    
londict

# criando novas colunas no dataframe

data['longitude_origem'] = data['sg_icao_origem'].apply(lambda x: londict.get(x))
data['longitude_destino'] = data['ICAO Aeródromoo Destino'].apply(lambda x: londict.get(x))
data